In [1]:
import pandas as pd
import numpy as np
import os

transferdir = "transfer/"

def process(text: str):
    from parse import compile
    from string import punctuation

    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans("", "", punctuation)).lower())
    return " ".join(out_list)

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df = df.rename(columns={"path":"hashname"})
df = df.loc[df.split.isin("test,dev".split(",")), :]

df["sentence"] = df.words.apply(" ".join).apply(process)
df["path"] = df.hashname.apply(lambda s: os.path.join(transferdir, s))

In [16]:

model_dir = "./38_with_lm/wav2vec2-xls-r-parlaspeech-hr/"
from transformers import AutoProcessor, Wav2Vec2ForCTC, AutoModelForCTC, Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)

Only 0 unigrams passed as vocabulary. Is this small or artificial data?


In [17]:

def get_transcript(audio_filepath:str):
    import soundfile as sf
    import torch

    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()
    inputs = processor(speech, sampling_rate=sample_rate, return_tensors="pt")
    with torch.no_grad():
      logits = model(**inputs).logits
    transcription = processor.batch_decode(logits.numpy()).text

    return transcription[0]

In [18]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
from tqdm.auto import tqdm
df["predictions"] = [get_transcript(path) for path in tqdm(df.path.values)]
for splt in ["dev", "test"]:
    print("Evaluating on ", splt)
    wer = wer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    cer = cer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    print(f"{wer=:0.4f}, {cer=:0.4f}")

  0%|          | 0/1013 [00:00<?, ?it/s]

Evaluating on  dev
wer=0.1129, cer=0.0448
Evaluating on  test
wer=0.0985, cer=0.0363


In [22]:
df[["sentence", "predictions", "split", "hashname"]].to_csv(
    "39_model_with_LM_output_for_evaluation.csv", 
    index=False
    )